In [1]:
import pipit
import stumpy
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import os 
os.environ["BOKEH_ALLOW_WS_ORIGIN"] = "127.0.0.1:8889"

In [3]:
trace = pipit.Trace.from_otf2(
    "/mnt/d/work/pipit-data/tortuga-otf2-16"
)
trace.calc_exc_metrics()
df = trace.events

iterations = len(
    df[
        (df["Name"] == "time-loop")
        & (df["Event Type"] == "Enter")
        & (df["Process"] == 0)
    ]
)

first_loop_enter = df[
    (df["Name"] == "time-loop")
    & (df["Event Type"] == "Enter")
    & (df["Process"] == 0)
].index[0]

last_loop_leave = df[
    (df["Name"] == "time-loop")
    & (df["Event Type"] == "Leave")
    & (df["Process"] == 0)
].index[-1]

data = df.iloc[first_loop_enter + 1 : last_loop_leave]
y = data.loc[(data["time.exc"].notnull()) & (data["Process"] == 0)]["time.exc"].values[:]
window_size = int(len(y) / iterations)

In [4]:
trace.detect_pattern(data, iterations=iterations, window_size=window_size, metric="time.exc")

,Timestamp (ns),Event Type,Name,Thread,Process,Attributes,_matching_event,_matching_timestamp,_depth,_parent,_children,time.inc,time.exc
4319,8.582351e+08,Enter,advanceTimestep_RK3,0,0,{'region': 'Region 96'},33886,1.016884e+09,2,4306,"[4335, 13212, 14123, 23088, 23953, 32919, 33803]",1.586485e+08,4.300941e+06
34335,1.022223e+09,Enter,advanceTimestep_RK3,0,0,{'region': 'Region 96'},63902,1.160285e+09,2,34307,"[34351, 43422, 44321, 53268, 54189, 63101, 63899]",1.380618e+08,4.083088e+06
64351,1.164424e+09,Enter,advanceTimestep_RK3,0,0,{'region': 'Region 96'},93918,1.302378e+09,2,64303,"[64367, 73456, 74419, 83281, 84201, 93109, 93915]",1.379545e+08,3.910133e+06


In [5]:
trace.plot_timeline()